# Tutorial: Bayesian Convolutional Neural Networks for Jet Classification

## Introduction

In this tutorial, you will learn to implement and train Bayesian Convolutional Neural Networks (Bayesian CNNs) for jet classification in particle physics. Bayesian neural networks provide uncertainty quantification by treating network weights as probability distributions rather than fixed values.

### Learning Objectives
- Understand the principles of Bayesian deep learning
- Implement Bayesian convolutional layers using the reparameterization trick
- Train a Bayesian CNN for binary classification of particle jets
- Quantify predictive uncertainty using Monte Carlo sampling
- Evaluate model performance with ROC curves and uncertainty analysis

### Background
Particle jets are collimated sprays of particles produced in high-energy collisions. Distinguishing between signal jets (e.g., from top quark decay) and background jets is crucial for many physics analyses. Bayesian neural networks not only provide classification predictions but also quantify uncertainty in those predictions, which is valuable for physics applications where understanding model confidence is essential.

### Tutorial Outline

1. **Data Loading and Preprocessing**
   - Load jet constituent data
   - Convert particle constituents to image representation
   - Preprocess jet images (centering, rotation, normalization)

2. **Bayesian Neural Network Components**
   - Implement variational Bayesian linear layers
   - Build Bayesian convolutional layers using reparameterization trick
   - Understand KL divergence regularization

3. **Model Architecture**
   - Design Bayesian CNN for jet classification
   - Combine Bayesian and standard layers

4. **Training and Evaluation**
   - Implement Bayesian loss function (negative log-likelihood + KL divergence)
   - Train the model with proper regularization
   - Evaluate classification performance

5. **Uncertainty Quantification**
   - Use Monte Carlo sampling for uncertainty estimation
   - Analyze prediction uncertainty patterns
   - Visualize uncertainty vs prediction confidence

## 1. Data Loading and Setup

### Load Required Libraries

In [ ]:
import os
import sys
import random
import time
import math
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import Tensor
from torch.nn.parameter import Parameter
from torch.nn import functional as F
from torch.nn import init, Module
from sklearn.metrics import roc_curve, roc_auc_score
from matplotlib import pyplot as plt

### Load Dataset

Load the data using the correct path you set previously for the script

In [ ]:
!pip install wget
!python3 data/get_data.py 2 data

In [ ]:
X_train = np.load( "data/toptagging-short/x_train_short.npy")
y_train = np.load( "data/toptagging-short/y_train_short.npy")
X_test = np.load( "data/toptagging-short/x_test_short.npy")
y_test = np.load( "data/toptagging-short/y_test_short.npy")
X_val = np.load( "data/toptagging-short/x_val_short.npy")
y_val = np.load( "data/toptagging-short/y_val_short.npy")

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

## 2. Data Preprocessing

The preprocessing functions are provided to convert particle constituents into images.

In [ ]:
# Grid settings for image creation
xpixels = np.arange(-2.6, 2.6, 0.029)  # eta range
ypixels = np.arange(-np.pi, np.pi, 0.035)  # phi range

def eta(pT, pz):
    """Calculate pseudorapidity from transverse momentum and z-momentum"""
    small = 1e-10
    small_pT = (np.abs(pT) < small)
    small_pz = (np.abs(pz) < small)
    not_small = ~(small_pT | small_pz)
    theta = np.arctan(pT[not_small]/pz[not_small])
    theta[theta < 0] += np.pi
    etas = np.zeros_like(pT)
    etas[small_pz] = 0
    etas[small_pT] = 1e-10
    etas[not_small] = -np.log(np.tan(theta/2))
    return etas

def phi(px, py):
    """Calculate azimuthal angle from x and y momentum components"""
    phis = np.arctan2(py, px)
    phis[phis < 0] += 2*np.pi
    phis[phis > 2*np.pi] -= 2*np.pi
    phis = phis - np.pi 
    return phis

def mass(E, px, py, pz):
    """Calculate invariant mass from 4-momentum"""
    return np.sqrt(np.maximum(0., E**2 - px**2 - py**2 - pz**2))

def img_mom(x, y, weights, x_power, y_power):
    """Calculate image moments for centering and rotation"""
    return ((x**x_power) * (y**y_power) * weights).sum()

def orig_image(etas, phis, es):
    """Create 2D histogram image from eta, phi coordinates and energies"""
    z = np.zeros((etas.shape[0], len(xpixels), len(ypixels)))
    in_grid = ~((etas < xpixels[0]) | (etas > xpixels[-1]) | 
                (phis < ypixels[0]) | (phis > ypixels[-1]))
    
    xcoords = np.argmin(np.abs(etas[:, None, :] - xpixels[None, :, None]), axis=1)
    ycoords = np.argmin(np.abs(phis[:, None, :] - ypixels[None, :, None]), axis=1)
    ncoords = np.repeat(np.arange(etas.shape[0])[:, None], etas.shape[1], axis=1)
    
    z[ncoords[in_grid], ycoords[in_grid], xcoords[in_grid]] = es[in_grid]
    return z

def preprocessing(x, y, weights, rotate=True, flip=True):
    """Preprocess jet image: center, rotate, and flip for standardization"""
    # Center the image
    x_centroid = img_mom(x, y, weights, 1, 0) / weights.sum()
    y_centroid = img_mom(x, y, weights, 0, 1) / weights.sum()
    x = x - x_centroid
    y = y - y_centroid

    if rotate:
        # Calculate covariance matrix for rotation
        u11 = img_mom(x, y, weights, 1, 1) / weights.sum()
        u20 = img_mom(x, y, weights, 2, 0) / weights.sum()
        u02 = img_mom(x, y, weights, 0, 2) / weights.sum()
        cov = np.array([[u20, u11], [u11, u02]])

        # Find principal axis
        evals, evecs = np.linalg.eig(cov)
        sort_indices = np.argsort(evals)[::-1]
        e_1 = evecs[:, sort_indices[0]]

        # Rotate so principal axis is vertical
        theta = np.arctan2(e_1[0], e_1[1])
        rotation = np.matrix([[np.cos(theta), -np.sin(theta)], 
                             [np.sin(theta), np.cos(theta)]])
        transformed_mat = rotation * np.stack([x, y])
        x_rot, y_rot = transformed_mat.A
    else: 
        x_rot, y_rot = x, y
  
    # Flip for consistency
    if flip:
        if weights[x_rot < 0.].sum() < weights[x_rot > 0.].sum():
            x_rot = -x_rot
        if weights[y_rot < 0.].sum() > weights[y_rot > 0.].sum():
            y_rot = -y_rot
            
    return x_rot, y_rot

def constit_to_img(jets, n_constit, norm, rotate, flip):
    """Convert jet constituents to preprocessed images"""
    print("Crop constituents")
    jets = jets[:, 0:n_constit, :]
    
    print("Calculating kinematic variables")
    E = jets[:, :, 0]
    pxs = jets[:, :, 1]
    pys = jets[:, :, 2]
    pzs = jets[:, :, 3]
    pT = np.sqrt(pxs**2 + pys**2)
    
    etas = eta(pT, pzs)
    phis = phi(pxs, pys)
    
    # Pre-shift phis to handle periodicity
    print("Pre-shifting phis")
    phis = (phis.T - phis[:, 0]).T
    phis[phis < -np.pi] += 2*np.pi
    phis[phis > np.pi] -= 2*np.pi
    
    weights = pT  # Use pT as weights
    
    print("Preprocessing")
    for i in range(np.shape(etas)[0]):
        etas[i, :], phis[i, :] = preprocessing(etas[i, :], phis[i, :], 
                                              weights[i, :], rotate, flip)
    
    print("Creating images")
    z_ori = orig_image(etas, phis, weights)
        
    print("Cropping and normalizing")
    n_crop = 40
    z_new = np.zeros((z_ori.shape[0], n_crop, n_crop))
    for i in range(z_ori.shape[0]):
        Npix = z_ori[i, :, :].shape
        z_new[i, :, :] = z_ori[i, 
                               int(Npix[0]/2 - n_crop/2):int(Npix[0]/2 + n_crop/2), 
                               int(Npix[1]/2 - n_crop/2):int(Npix[1]/2 + n_crop/2)]
        if norm:
            z_sum = z_new[i, :, :].sum()
            if z_sum != 0.:
                z_new[i, :, :] = z_new[i, :, :] / z_sum
    
    print("Reshaping")
    z_out = z_new.reshape((z_new.shape[0], -1))
    
    return z_out

### Convert Constituents to Images

In [ ]:
# Convert constituent data to images
z_train = constit_to_img(X_train, 50, True, True, True)
z_test = constit_to_img(X_test, 50, True, True, True)
z_val = constit_to_img(X_val, 50, True, True, True)

### Visualize Average Jet Images

In [ ]:
# Separate signal and background for visualization
sig = z_train[np.where(y_train == 1)]
bkg = z_train[np.where(y_train == 0)]

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Plot average signal jet
axs[0].imshow(sig.mean(0).reshape((40, 40)), cmap="gist_heat_r")
axs[0].set_title("Average Signal Jet (Top)")
axs[0].set_xlabel("$\\eta$")
axs[0].set_ylabel("$\\phi$")

# Plot average background jet
axs[1].imshow(bkg.mean(0).reshape((40, 40)), cmap="gist_heat_r")
axs[1].set_title("Average Background Jet (QCD)")
axs[1].set_xlabel("$\\eta$")
axs[1].set_ylabel("$\\phi$")

plt.tight_layout()
plt.show()

## 3. Dataset and DataLoader Setup

In [ ]:
class CNNData(Dataset):
    """Custom dataset for CNN training"""
    
    def __init__(self, imgs, labels):
        self.imgs = imgs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.imgs[idx], self.labels[idx]

In [ ]:
# Reshape for CNN input and convert to tensors
z_train = torch.Tensor(z_train.reshape(-1, 1, 40, 40).astype('float32'))
z_test = torch.Tensor(z_test.reshape(-1, 1, 40, 40).astype('float32'))
z_val = torch.Tensor(z_val.reshape(-1, 1, 40, 40).astype('float32'))

y_train = torch.Tensor(y_train).unsqueeze(-1)
y_test = torch.Tensor(y_test).unsqueeze(-1)
y_val = torch.Tensor(y_val).unsqueeze(-1)

# Create datasets and dataloaders
train_dataset = CNNData(z_train, y_train.float())
test_dataset = CNNData(z_test, y_test.float())
val_dataset = CNNData(z_val, y_val.float())

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

print(f"Data shapes - Train: {z_train.shape}, Val: {z_val.shape}, Test: {z_test.shape}")

## 4. Bayesian Neural Network Components

### Variational Bayesian Linear Layer

We give the code for a variational Bayesian linear layer, as seen in the tutorial for bayesian regression.

**Key concepts:**
- Weights are distributions: w ~ N(μ, σ²)
- Reparameterization trick: w = μ + σ * ε, where ε ~ N(0,1)
- KL divergence measures distance from prior

In [ ]:
class VBLinear(Module):
    """Variational Bayesian Linear Layer
    
    Implements a linear layer where weights are treated as distributions.
    Uses reparameterization trick: w = μ + σ * ε, where ε ~ N(0,1)
    """
    
    def __init__(self, in_features, out_features):
        super(VBLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.resample = True
        
        # Parameters for weight distribution: w ~ N(μ, σ²)
        self.bias = Parameter(Tensor(out_features))
        self.mu_w = Parameter(Tensor(out_features, in_features))  # Weight means
        self.logsig2_w = Parameter(Tensor(out_features, in_features))  # Log weight variances
        
        # Random noise for reparameterization
        self.random = torch.randn_like(self.logsig2_w)
        self.reset_parameters()
        
    def forward(self, input):
        if self.resample:
            self.random = torch.randn_like(self.logsig2_w)
        
        # Reparameterization trick: w = μ + σ * ε
        s2_w = self.logsig2_w.exp()  # Convert log variance to variance
        weight = self.mu_w + s2_w.sqrt() * self.random
        
        return F.linear(input, weight, self.bias)
    
    def reset_parameters(self):
        """Initialize parameters"""
        stdv = 1. / np.sqrt(self.mu_w.size(1))
        self.mu_w.data.normal_(0, stdv)
        self.logsig2_w.data.zero_().normal_(-9, 0.001)  # Start with small variance
        self.bias.data.zero_()
        
    def KL(self, loguniform=False):
        """Calculate KL divergence from prior N(0,1)"""
        return 0.5 * (self.mu_w.pow(2) + self.logsig2_w.exp() - self.logsig2_w - 1).sum()

### Bayesian Convolutional Layer

Implement the BBBConv2d class, which is a Bayesian version of the Conv2d layer. 

In [ ]:
from torch.nn.common_types import _size_2_t
from typing import Optional, Union

def KL_DIV(mu_q, sig_q, mu_p, sig_p):
    """Calculate KL divergence between two Gaussians"""
    kl = 0.5 * (2 * torch.log(sig_p / sig_q) - 1 + 
                (sig_q / sig_p).pow(2) + ((mu_p - mu_q) / sig_p).pow(2)).sum()
    return kl

class BBBConv2d(Module):
    """Bayes by Backprop Convolutional Layer"""
    
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, dilation=1, bias=True, priors=None):
        super(BBBConv2d, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size if isinstance(kernel_size, tuple) else (kernel_size, kernel_size)
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = 1
        self.use_bias = bias
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Prior and posterior parameters
        if priors is None:
            priors = {
                'prior_mu': 0,
                'prior_sigma': 0.1,
                'posterior_mu_initial': (0, 0.1),
                'posterior_rho_initial': (-3, 0.1),
            }
        self.prior_mu = priors['prior_mu']
        self.prior_sigma = priors['prior_sigma']
        self.posterior_mu_initial = priors['posterior_mu_initial']
        self.posterior_rho_initial = priors['posterior_rho_initial']

        # Weight parameters: μ and ρ (where σ = log(1 + exp(ρ)))
        #TODO: Implement the mu and rho parameters for weights
        self.W_mu = # YOUR CODE HERE
        self.W_rho = # YOUR CODE HERE

        if self.use_bias:
            self.bias_mu = Parameter(torch.empty((out_channels), device=self.device))
            self.bias_rho = Parameter(torch.empty((out_channels), device=self.device))
        else:
            self.register_parameter('bias_mu', None)
            self.register_parameter('bias_rho', None)

        self.reset_parameters()

    def reset_parameters(self):
        """Initialize parameters"""
        self.W_mu.data.normal_(*self.posterior_mu_initial)
        self.W_rho.data.normal_(*self.posterior_rho_initial)

        if self.use_bias:
            self.bias_mu.data.normal_(*self.posterior_mu_initial)
            self.bias_rho.data.normal_(*self.posterior_rho_initial)

    def forward(self, input, sample=True):
        """Forward pass with weight sampling"""
        if self.training or sample:
            # Sample weights: w = μ + σ * ε
            #TODO: Implement the sampling of weights
            W_eps = # YOUR CODE HERE
            self.W_sigma = # YOUR CODE HERE: Compute σ = log(1 + exp(ρ))
            weight = None # YOUR CODE HERE: Compute w = μ + σ * ε

            if self.use_bias:
                bias_eps = torch.empty(self.bias_mu.size()).normal_(0, 1).to(self.device)
                self.bias_sigma = torch.log1p(torch.exp(self.bias_rho))
                bias = self.bias_mu + bias_eps * self.bias_sigma
            else:
                bias = None
        else:
            # Use mean weights for deterministic inference
            weight = # YOUR CODE HERE
            bias = # YOUR CODE HERE if self.use_bias else None

        return F.conv2d(input, weight, bias, self.stride, 
                       self.padding, self.dilation, self.groups)

    def KL(self):
        """Calculate KL divergence from prior"""
        kl = KL_DIV(self.prior_mu, self.prior_sigma, self.W_mu, self.W_sigma)
        if self.use_bias:
            kl += KL_DIV(self.prior_mu, self.prior_sigma, self.bias_mu, self.bias_sigma)
        return kl

## 5. Bayesian CNN Architecture

**Exercise**: Complete the Bayesian CNN architecture. Pay attention to which layers are Bayesian and which are standard.

In [ ]:
class BayesConvNet2D(nn.Module):
    """Bayesian Convolutional Neural Network for Jet Classification"""
    
    def __init__(self, training_size=None, in_ch=1, ch=4, out_dim=1, img_sz=40):
        super().__init__()
        self.training_size = training_size
        
        # Track Bayesian layers for KL calculation
        self.vb_layers = []
        self.all_layers = []
        
        # TODO: First Bayesian convolutional block
        self.conv1 = # YOUR CODE HERE - BBBConv2d layer
        self.vb_layers.append(self.conv1)
        self.all_layers.append(self.conv1)
        self.all_layers.append(nn.ReLU())
        
        # First pooling layer
        self.max1 = nn.MaxPool2d(kernel_size=2)
        self.all_layers.append(self.max1)
        self.all_layers.append(nn.ReLU())
        
        # TODO: Second Bayesian convolutional block
        self.conv2 = # YOUR CODE HERE - BBBConv2d layer
        self.vb_layers.append(self.conv2)
        self.all_layers.append(self.conv2)
        
        # Second pooling layer
        self.max2 = nn.MaxPool2d(kernel_size=2)
        self.all_layers.append(self.max2)
        
        # Flatten for fully connected layer
        self.flatten = nn.Flatten()
        self.all_layers.append(self.flatten)
        
        # TODO: Bayesian fully connected output layer
        self.out = # YOUR CODE HERE - VBLinear layer
        self.vb_layers.append(self.out)
        self.all_layers.append(self.out)
        
        # Sigmoid activation for binary classification
        self.all_layers.append(nn.Sigmoid())
    
    def forward(self, x):
        """Forward pass through the network"""
        self.net = nn.Sequential(*self.all_layers)
        return self.net(x)
    
    def KL(self):
        """Calculate total KL divergence from all Bayesian layers"""
        # TODO: Sum KL divergence from all Bayesian layers
        kl = 0
        for vb_layer in self.vb_layers:
            kl += # YOUR CODE HERE
        return kl / self.training_size  # Normalize by training set size

### Test the Model

In [ ]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Test the network with a single sample
test_net = BayesConvNet2D()
sample_input = train_dataset[0][0].unsqueeze(0)
sample_output = test_net(sample_input)
print(f"Sample input shape: {sample_input.shape}")
print(f"Sample output: {sample_output.item():.4f}")
print(f"Sample KL divergence: {test_net.KL().item():.6f}")

## 6. Training Functions

**Exercise**: Complete the training function. The key is implementing the Bayesian loss = NLL + KL divergence.

In [ ]:
def train_epoch(dataloader, model, optimizer):
    """Train for one epoch"""
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        # Forward pass
        pred = model(X)
        
        # TODO: Calculate Bayesian loss
        loss_fn = nn.BCELoss()
        nll = # YOUR CODE HERE - Negative log-likelihood
        kl = # YOUR CODE HERE - KL divergence regularization
        loss = # YOUR CODE HERE - Total Bayesian loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print progress
        if batch % 100 == 0:
            current = batch * len(X)
            print(f"Loss: {loss:>8f} KL: {kl:>8f} NLL: {nll:>8f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def evaluate_model(dataloader, model, dataset_name=""):
    """Evaluate model on a dataset"""
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_nll, total_kl, total_loss = 0.0, 0.0, 0.0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            
            loss_fn = nn.BCELoss()
            nll = loss_fn(pred, y)
            kl = model.KL()
            loss = nll + kl
            
            total_nll += nll.item()
            total_kl += kl.item()
            total_loss += loss.item()

    avg_nll = total_nll / num_batches
    avg_kl = total_kl / num_batches
    avg_loss = total_loss / num_batches
    
    print(f"{dataset_name} Loss: {avg_loss:>8f} KL: {avg_kl:>8f} NLL: {avg_nll:>8f}")
    return avg_nll, avg_kl, avg_loss

## 7. Model Training

**Exercise**: Set up and run the training loop.

In [ ]:
def separator():
    print("-" * 60)

# Training hyperparameters
epochs = 50
learning_rate = 5e-4
trn_len = len(y_train)

print(f"Training dataset size: {trn_len}")

# TODO: Initialize model and optimizer
model = # YOUR CODE HERE
optimizer = # YOUR CODE HERE

separator()
print("Model Architecture:")
separator()
print(model)
separator()

In [ ]:
# Training loop
train_losses, val_losses = [], []
train_nlls, val_nlls = [], []
train_kls, val_kls = [], []

for epoch in range(epochs):
    separator()
    print(f"Epoch {epoch + 1}/{epochs}")
    separator()
    
    # Training
    train_epoch(train_dataloader, model, optimizer)
    
    # Evaluation
    train_nll, train_kl, train_loss = evaluate_model(train_dataloader, model, "Train")
    val_nll, val_kl, val_loss = evaluate_model(val_dataloader, model, "Val")
    
    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_nlls.append(train_nll)
    val_nlls.append(val_nll)
    train_kls.append(train_kl)
    val_kls.append(val_kl)

print("\nTraining completed!")

## 8. Results Analysis

**Exercise**: Implement Monte Carlo sampling for uncertainty quantification.

In [ ]:
def get_prediction(x, model, n_monte=20):
    """Get predictions with uncertainty using Monte Carlo sampling"""
    model.eval()
    predictions = []
    
    for i in range(n_monte):
        print(f"Monte Carlo sample {i+1}/{n_monte}")
        with torch.no_grad():
            # TODO: Get prediction and extract first column
            pred = # YOUR CODE HERE
            predictions.append(pred)
        
    predictions = torch.stack(predictions)
    # TODO: Calculate mean and standard deviation
    mean = # YOUR CODE HERE
    std = # YOUR CODE HERE
    
    return mean, std

In [ ]:
# Evaluate on test set with uncertainty quantification
samp_size = 2000  # Use subset for faster evaluation

# Get predictions with uncertainty
test_pred, test_std = get_prediction(z_test[:samp_size], model, n_monte=10)

# Calculate accuracy
test_pred_labels = torch.round(test_pred[:samp_size])
test_correct = (test_pred_labels == y_test[:samp_size]).sum().item()
test_accuracy = test_correct / len(test_pred_labels) * 100

print(f"Test Accuracy: {test_accuracy:.1f}%")

### Plot Training Curves

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

# Total loss
axs[0].plot(train_losses, label="Train Loss", color='tab:red')
axs[0].plot(val_losses, label="Val Loss", color='tab:green')
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Total Loss")
axs[0].legend()
axs[0].grid(True)

# Negative log-likelihood
axs[1].plot(train_nlls, label="Train NLL", color='tab:red')
axs[1].plot(val_nlls, label="Val NLL", color='tab:green')
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Negative Log-Likelihood")
axs[1].legend()
axs[1].grid(True)

# KL divergence
axs[2].plot(train_kls, label="Train KL", color='tab:red')
axs[2].plot(val_kls, label="Val KL", color='tab:green')
axs[2].set_xlabel("Epoch")
axs[2].set_ylabel("KL Divergence")
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()

### ROC Analysis and Uncertainty Visualization

**Exercise**: Complete the uncertainty analysis.

In [ ]:
# Calculate ROC metrics
fpr, tpr, thresholds = roc_curve(y_test[:samp_size].numpy(), test_pred.numpy())
auc_score = roc_auc_score(y_test[:samp_size].numpy(), test_pred.numpy())

print(f"AUC Score: {auc_score:.3f}")

# TODO: Analyze uncertainty patterns
test_std_np = # YOUR CODE HERE - Convert uncertainty to numpy
test_pred_np = # YOUR CODE HERE - Convert predictions to numpy

fig, ax = plt.subplots(1, 3, figsize=(18, 5))

# TODO: Plot prediction distribution
# TODO: Plot uncertainty distribution  
# TODO: Plot uncertainty vs prediction scatter plot

plt.tight_layout()
plt.show()

print(f"\nUncertainty Statistics:")
print(f"Mean uncertainty: {test_std_np.mean():.4f}")
print(f"Max uncertainty: {test_std_np.max():.4f}")
print(f"Min uncertainty: {test_std_np.min():.4f}")

## 10. Summary and Conclusions

In this tutorial, you have learned:

1. **Bayesian Deep Learning Fundamentals**: How to treat neural network weights as probability distributions rather than fixed values.

2. **Implementation Details**: 
   - Variational Bayesian layers using the reparameterization trick
   - Bayesian convolutional layers for image processing
   - KL divergence regularization for proper weight distribution learning

3. **Training Bayesian Networks**: How to combine negative log-likelihood with KL divergence for the total loss function.

4. **Uncertainty Quantification**: Using Monte Carlo sampling to estimate predictive uncertainty, which is crucial for physics applications.

5. **Performance Evaluation**: Standard classification metrics (ROC curves, AUC) combined with uncertainty analysis.

### Key Advantages of Bayesian CNNs:
- **Uncertainty Quantification**: Know when the model is uncertain
- **Regularization**: KL divergence acts as automatic regularization
- **Robustness**: Better handling of out-of-distribution samples

### Applications in Particle Physics:
- **Systematic Uncertainty Estimation**: Model uncertainty contributes to systematic uncertainties
- **Data Quality Assessment**: High uncertainty regions may indicate problematic data
- **Decision Making**: Uncertainty-aware decisions in analysis pipelines

This framework can be extended to other particle physics applications such as particle identification, trigger systems, and analysis-specific classifiers where uncertainty quantification is valuable.